### 1.1 Extract Information From Your Own Transaction

In [ ]:
import requests
import datetime
import json

# '826534' is the block height
block_height = 826534
url = f'https://api.whatsonchain.com/v1/bsv/main/block/height/{block_height}'

response = requests.get(url)
block_data = response.json()

# Extracting
txcount = block_data['txcount']
time = block_data['time']
totalFees = block_data['totalFees']
confirmations = block_data['confirmations']
miner = block_data['miner']

# Convert the Unix timestamp
timestamp = datetime.datetime.fromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')

# Print the data in JSON format
print(json.dumps({
    'txcount': txcount,
    'time': timestamp,
    'totalFees': totalFees,
    'confirmations': confirmations,
    'miner': miner
}, indent=4))

#### Here's what each of these parts of the block represents:

**txcount:** This is the number of transactions included in the block. Each transaction represents a transfer of BSV (Bitcoin Satoshi Vision) from one account to another.

**time:** This is the Unix timestamp for when the block was mined. It indicates the moment the block was added to the blockchain. The code converts this timestamp into a human-readable date and time format.

**totalFees:** This is the total amount of fees collected by the miner for including transactions in this block. Users pay transaction fees as an incentive for miners to include their transactions in blocks.

**confirmations:** This number indicates how many blocks have been added to the blockchain since the block in question was mined. The more confirmations there are, the more secure the transaction is considered to be.

**miner:** This refers to the identifier of the miner or mining pool that successfully mined this block. It is usually a representation of the miner's address or a name if the mining pool has one.

### 2.1 Obtain Time Series Data

In [ ]:
import yfinance as yf
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import datetime


end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=365)

# High-risk asset: Bitcoin
btc = yf.download('BTC-USD', start=start_date, end=end_date)
# Low-risk asset: Gold price index
gold = web.DataReader('ID7108', 'fred', start=start_date, end=end_date)
# Overall stock market performance: S&P 500
sp500 = yf.download('^GSPC', start=start_date, end=end_date)

plt.figure(figsize=(14,7))

# Plot Bitcoin prices
plt.subplot(3, 1, 1)
plt.plot(btc['Close'], label='Bitcoin (BTC)')
plt.title('Bitcoin Price')
plt.ylabel('Price (USD)')
plt.xlabel('Date')
plt.legend()

# Plot Gold prices
plt.subplot(3, 1, 2)
plt.plot(gold, label='Gold Price Index (ID7108)')
plt.title('Gold Price Index')
plt.ylabel('Index Value')
plt.xlabel('Date')
plt.legend()

# Plot S&P 500 prices
plt.subplot(3, 1, 3)
plt.plot(sp500['Close'], label='S&P 500')
plt.title('S&P 500 Index')
plt.ylabel('Index Value')
plt.xlabel('Date')
plt.legend()


plt.tight_layout()
plt.show()

### 2.2 Data Transformations

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

btc = yf.download('BTC-USD', period="max")['Close']
gold = yf.download('GC=F', period="max")['Close'] 
sp500 = yf.download('^GSPC', period="max")['Close']

data = pd.DataFrame({
    'BTC': btc,
    'GOLD': gold,
    'SP500': sp500
})

# Drop NaN
data.dropna(inplace=True)

# Transform observations
returns = np.log(data / data.shift(1))
# Drop NaN
returns.dropna(inplace=True)

print(returns)

### 2.3 Data Analysis

In [ ]:
import statsmodels.api as sm


X = sm.add_constant(returns['SP500'])  # Market returns
y = returns['BTC']  # Bitcoin returns

# Perform OLS regression
model = sm.OLS(y, X).fit()

model_summary = model.summary()
print(model_summary)

#### Analysis
**1. Regression results:**

- **Alpha (const):** The coefficient for the constant is 0.0017 with a standard error of 0.001. The t-statistic is 1.857, and the p-value is 0.063. 

- **Beta (SP500):** The beta coefficient is 0.8298 with a standard error of 0.078. The t-statistic is 10.644, which is highly significant with a p-value effectively at 0.000. 

- **R-squared:** The R-squared value is 0.046, and the adjusted R-squared is also 0.046. 

- **F-statistic:** The F-statistic is 113.3 and the associated p-value is 7.19e-26, suggesting that the model is statistically significant.

**2. Interpretation in the context of CAPM:**

- **Beta Interpretation:** A beta of less than 1 indicates that Bitcoin is less volatile than the broader market, which is somewhat contrary to general market observations, as Bitcoin is often considered more volatile. This could be due to a variety of factors, including the time frame of the data, specific market events, or the nature of the investors involved.

- **Alpha Interpretation:** The alpha is not statistically significant at the 5% level, but it is borderline significant. This means that there isn't strong evidence that Bitcoin's returns are significantly different from what CAPM would predict given its systematic risk, as measured by beta. However, the positive alpha points to a tendency for Bitcoin to outperform the expectations of CAPM during the period analyzed.

- **Overall Model Interpretation:** The low R-squared suggests that market returns are not a strong predictor of Bitcoin returns. This is consistent with the understanding that Bitcoin's price movements are driven by a range of factors that are not necessarily linked to traditional equity markets.

### 3.1 High Level Description of FinTech Firm

Sarunas' FinTech firm is structured as a tech-driven enterprise offering loan services to individuals. The project is modular, with four primary structural components that interact to automate and streamline the loan application and approval process. Here’s a mechanical description of each part and their interactions:

1. **Streamlit UI Service**:
   - **Purpose**: This is the user interface where potential customers can submit their loan applications. The UI is designed to be intuitive and accessible, allowing users to input their personal and financial information securely.
   - **Interaction**: Upon submission, the application data is sent to the Statistical Model for processing.
   - **Achievement**: It serves as the entry point for customers, simplifying the application process and enhancing user experience.

2. **Statistical Model Service**:
   - **Purpose**: This service hosts a machine learning model, specifically a logistic regression model, that evaluates the risk associated with a loan application. Logistic regression is chosen for its ability to provide probabilities that an event will happen (e.g., loan default) based on input variables from the application.
   - **Interaction**: It receives data from the Streamlit UI, processes it, and then sends the result to the Postgres Database for storage. It may also interact with JupyterLab for model training and analysis.
   - **Achievement**: By predicting the likelihood of loan repayment, it automates the decision-making process, thus allowing the firm to issue loans with an informed understanding of the risk involved.

3. **Postgres Database**:
   - **Purpose**: This database is where all the requests and their corresponding responses are stored.
   - **Interaction**: The database receives processed data from the Statistical Model and serves as a repository for storing and retrieving application data.
   - **Achievement**: It ensures that all transaction data is securely managed and can be accessed for reporting, auditing, or further analysis.

4. **JupyterLab Service**:
   - **Purpose**: JupyterLab is used for data analysis and exploration. It allows data scientists and analysts to work with the data, build models, and test hypotheses in an interactive environment.
   - **Interaction**: It can access the Postgres Database to retrieve data for analysis and is the environment where the logistic regression model is likely developed and refined before being deployed to the Statistical Model service.
   - **Achievement**: It is the research and development component, providing insights that drive business strategy and improve the model's performance.

**Logistic Regression** is a statistical method for predicting binary outcomes. It’s useful for scenarios where the outcome is dichotomous (e.g., loan default: yes or no). Its output is a probability that the given input point belongs to a certain class, which translates well into risk assessment for loan approval. This technique is useful because it is relatively simple, interpretable, and efficient for binary classification problems. In the context of Sarunas' FinTech firm, logistic regression helps in making informed loan approval decisions by assessing the probability of a borrower defaulting on a loan.